# Testing AWS

If you've ever tried to test the use of AWS in code before you know there are quite a few issues:<br>
- There isn't a clear object to mock
- Even if you have a wrapper/wrapper library to mock: 
 - there isn't a good way to test the code inside the wrapper (let's say you provided `bucket=` instead of `Bucket=`)
 - you can't always be sure the results you expected to get and mocked up actually match the reality
- There is localstack, but that takes a time to spin up and eats up resources locally (+ ugly docker-compose files)

This often leads to untested or poorly tested code. Existing mocks are often difficult to understand and quickly get more and more convoluted. 

## Moto saves the day
So here comes a library actively developed and supported by many. It mocks out the internals of AWS in a neat and standardized way, allowing you to interact with the AWS as if nothing was mocking it. It goes through all the normal loops and hoops of botocore and provides realistic responses.<br>
<br>
While we'll go over some of it here, in case you ever want a clearer explanation or more detail here is the [documentation](http://docs.getmoto.org/en/latest/docs/getting_started.html#decorator) and the [source code](https://github.com/spulec/moto). 

## Example of a test (as close as we can get in this format)

So here is a tiny function we want to test

In [48]:
import boto3

def some_upload(data: bytes, bucket: str, key: str):
    client = boto3.client("s3")
    client.put_object(Body=data, Bucket=bucket, Key=key)

Let's create a test for it. Using pytest within jupyterbook doesn't seem straightforward, but I'll try to follow the normal structure. 

In [49]:
from moto import mock_s3

@mock_s3  # Here for safety - more info later
#pytest.fixture()
def bucket(client):
    bucket_name = "test_bucket"
    client.create_bucket(Bucket=bucket_name)
    
    return bucket_name
    # In a real fixture we would yield and clean up here

@mock_s3
def test_upload():
    client = boto3.client("s3")
    bucket_name = bucket(client)
    
    data = b"Made up file contents because I have no imagination."
    key = "somewhere/something.txt"
    some_upload(data, bucket_name, key)  # Notice how we don't have to provide a mocked client object.
    
    # Check we have 1 object in the bucket
    objects = client.list_objects(Bucket=bucket_name)
    assert len(objects["Contents"]) == 1
    
    # Check the contents
    uploaded_file = client.get_object(Bucket=bucket_name, Key=key)
    assert uploaded_file["Body"].read() == data

As this is a test, no output means success. That's a little boring, but let's make sure it didn't fail:

In [50]:
test_upload()

## Let's look inside and see those responses for ourselves

Now that we see that the test doesn't fail, it would be more fun to actually see the responses from the fake aws. Let's run all of these steps one by one and look at the aws responses

In [52]:
from pprint import pprint

mock = mock_s3()
mock.start()  # Can be used as a decorator, context manager, or raw - more on that later.

bucket_name = "test_bucket"
client = boto3.client("s3")
client.create_bucket(Bucket=bucket_name)
client.create_bucket(Bucket="some_bucket2")
client.create_bucket(Bucket="some_bucket3")
pprint(client.list_buckets())

{'Buckets': [{'CreationDate': datetime.datetime(2020, 10, 2, 3, 6, 5, 967829, tzinfo=tzutc()),
              'Name': 'test_bucket'},
             {'CreationDate': datetime.datetime(2020, 10, 2, 3, 6, 5, 969823, tzinfo=tzutc()),
              'Name': 'some_bucket2'},
             {'CreationDate': datetime.datetime(2020, 10, 2, 3, 6, 5, 972156, tzinfo=tzutc()),
              'Name': 'some_bucket3'}],
 'Owner': {'DisplayName': 'webfile', 'ID': 'bcaf1ffd86f41161ca5fb16fd081034f'},
 'ResponseMetadata': {'HTTPHeaders': {},
                      'HTTPStatusCode': 200,
                      'RetryAttempts': 0}}


In [53]:
data = b"Something something"
key = "somewhere/something.txt"
some_upload(data, bucket_name, key)
    
objects = client.list_objects(Bucket=bucket_name)
pprint(objects)

{'Contents': [{'ETag': '"50a39ec9e0e46cf2826eb5745e1c800b"',
               'Key': 'somewhere/something.txt',
               'LastModified': datetime.datetime(2020, 10, 2, 3, 6, 12, tzinfo=tzutc()),
               'Owner': {'DisplayName': 'webfile',
                         'ID': '75aa57f09aa0c8caeab4f8c24e99d10f8e7faeebf76c078efc7c6caea54ba06a'},
               'Size': 19,
               'StorageClass': 'STANDARD'}],
 'Delimiter': 'None',
 'IsTruncated': False,
 'MaxKeys': 1000,
 'Name': 'test_bucket',
 'ResponseMetadata': {'HTTPHeaders': {},
                      'HTTPStatusCode': 200,
                      'RetryAttempts': 0}}


In [54]:
uploaded_file = client.get_object(Bucket=bucket_name, Key=key)
pprint(uploaded_file)

{'Body': <botocore.response.StreamingBody object at 0x10e070a90>,
 'ContentLength': 19,
 'ETag': '"50a39ec9e0e46cf2826eb5745e1c800b"',
 'LastModified': datetime.datetime(2020, 10, 2, 3, 6, 12, tzinfo=tzutc()),
 'Metadata': {},
 'ResponseMetadata': {'HTTPHeaders': {'content-length': '19',
                                      'content-md5': 'UKOeyeDkbPKCbrV0XhyACw==',
                                      'etag': '"50a39ec9e0e46cf2826eb5745e1c800b"',
                                      'last-modified': 'Fri, 02 Oct 2020 '
                                                       '03:06:12 GMT'},
                      'HTTPStatusCode': 200,
                      'RetryAttempts': 0}}


In [55]:
print(uploaded_file["Body"].read())
mock.stop()

b'Something something'


## Notes I took while learning to use it: 
### How you use it
mock_X can be used as a decorator, context manager and raw object. Make an conscious choice when using them. My own notes on their use-cases:
- <b>decorator</b>: best for nice small tests that touch aws only inside the test itself. Also can be used as a precaution (see below). Don't use on aws fixtures - decorators and yield statements can get funky.
- <b>context manager</b>: best for fixtures and anywhere you want to have a very clear control over context's scope.
- <b>raw</b>: best for trying it out in a shell or presenting it to another step-by-step.

### Be mindful of your own mistakes
It seems like we (developers) are always able to access AWS without any extra steps (such as auth every so often or a VPN). While that's great, it also leaves us prone to easy oopsies: 
##### Oopsie example 1: 
There is a helper function for testing that touches AWS and relies on the context of the caller. Calling it outside of mocked context by accident might do something you didn't mean (ex: sent an email to everyone, uploaded a file into a production bucket or even worse - deleted everything inside a bucket)
##### Oopsie example 2: 
Maybe the helper isn't relying on the caller's context, but, for example, you accidentally deleted the `@mock_s3` line. This mistake isn't as easy to prevent as one from example 1, but the result might be as devastating. 

##### Keep yourself safe:
- Use a decorator on these methods <i>- don't rely on external context. Moto's works just as fine even if the contexts overlap.</i>
- Always override bucket names, paths, etc in the tests - <i>Don't let a default, an environment variable or .env file eventually drop you into a fire. Honestly, it's best to do that in any case. </i>
- If you're not sure, disconnect while you set up or make major changes - <i>The extra paranoid (so, basically, me) can always move the aws config temporarily or turn off the internet until the (unit) tests are properly set up.</i>